In [1]:
import os
import cv2
import math
from copy import deepcopy
import numpy as np
import pandas as pd
import imutils
from random import randint
from PIL import Image
from PIL.ImageShow import show

In [2]:
def DSLoad(dsFoderPath):
    filepathAnnot = dsFoderPath+"/allAnnotations.csv"
    dfAnnotation = pd.read_csv(filepathAnnot, sep=";")
    imgSize = []
    for index, row in dfAnnotation[1:].iterrows():
        image = cv2.imread(dsFoderPath+"/"+row[0])
        imgSize.append([image.shape[0], image.shape[1]])

    imgSizeDF = pd.DataFrame(imgSize, columns=["width", "height"])
    signUnique = dfAnnotation['Annotation tag'].unique()
    widthUnique = imgSizeDF["width"].unique().tolist()
    heightUnique = imgSizeDF["height"].unique().tolist()
    print(len(widthUnique), widthUnique)
    print(len(heightUnique), heightUnique)


In [3]:

def detectSign(img):
    try:
        grayScale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    except:
        grayScale = img
    try:
        bLFilter = cv2.bLFilter(grayScale, 11, 17, 17)
    except:
        bLFilter = grayScale
    imgMedian = np.median(img)
    thresholdLower = max(0, (0.7 * imgMedian))
    thresholdUpper = min(255, (0.7 * imgMedian))

    cannyEdged = cv2.Canny(bLFilter, thresholdLower, thresholdUpper)
    (cnts, _) = cv2.findContours(cannyEdged,
                                 cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:]
    maxArea = 0
    detectedSigns = []
    for c in cnts:
        perimeter = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
        if (len(approx) == 4):
            signArea = approx
            x, y, w, h = cv2.boundingRect(signArea)
            area = w*h
            if(area >= maxArea):
                maxArea = area
                if([x, y, w, h] not in detectedSigns):
                    detectedSigns.append([x, y, w, h])
    return detectedSigns


In [4]:
def SaveImage(name, img, displayImg = False) -> None:
    imge = Image.fromarray(img.astype(np.uint8))
    imge.save(name+".png")
    if(displayImg):
        show(imge, name)


In [5]:
def convertGrayScale(img, greyScalefilter = np.array([0.2989, 0.5870, 0.1140])):
    imgcopy = deepcopy(img)
    greyScaleImg = np.dot(imgcopy[:, :, :3], greyScalefilter)
    return greyScaleImg


In [6]:
def AddGaussianNoise(img, mean, std):
    noise = np.random.normal(mean, std, size=img.shape)
    imgNoisy = img + noise
    return imgNoisy


def AddSaltPepperNoise(img, percent):
    saltPepperImage = np.copy(img)
    saltPepperCount = (((img.shape[0]*img.shape[1])//100)*percent)//2
    for i in range(saltPepperCount):
        saltPepperPixel = [
            randint(0, img.shape[0])-1, randint(0, img.shape[1])-1]
        saltPepperImage[saltPepperPixel[0], saltPepperPixel[1]] = 0
        saltPepperPixel = [
            randint(0, img.shape[0])-1, randint(0, img.shape[1])-1]
        saltPepperImage[saltPepperPixel[0], saltPepperPixel[1]] = 255
    return saltPepperImage


In [7]:
def MedianFilter(img, k):
    imgOutput = np.zeros(img.shape)
    pad = (k//2)*2
    ImgPadded = np.zeros((img.shape[0]+pad, img.shape[1]+pad))
    ImgPadded[(pad//2):ImgPadded.shape[0] - (pad//2),
                (pad//2):ImgPadded.shape[1] - (pad//2)] = img
    noRows, noCols = img.shape
    for i in range(noRows):
        for j in range(noCols):
            try:
                imgOutput[i, j] = np.median(ImgPadded[i:i+k, j:j+k])
            except:
                print("error")
    return imgOutput


In [8]:
def FindNorm(k, std, mean = 0):
    denom = (math.sqrt(2 * math.pi)*std)
    expo = -((k**2) / 2*(std**2))
    return (math.e**expo)/denom


def GenerateGaussianKernel(size, std: int = 1):
    OneDKernel = np.linspace(-(size // 2), size // 2, size)
    for i in range(size):
        OneDKernel[i] = FindNorm(OneDKernel[i], std)
    twoDKernel = np.outer(OneDKernel, OneDKernel)
    twoDKernel *= 1.0 / twoDKernel.max()
    return twoDKernel


def GenerateBoxKernel(size):
    boxFilter = np.zeros((size, size))
    boxFilter += 1
    boxFilter = boxFilter / np.sum(boxFilter)
    return boxFilter


In [9]:
def Convolve(img, kernel):
    imgOutput = np.zeros(img.shape)
    noRows, noCols = img.shape
    noKRows, noKCols = kernel.shape
    padHeight, padWidth = (noKRows//2)*2, (noKCols//2)*2
    ImgPadded = np.zeros((noRows + padHeight, noCols + padWidth))
    ImgPadded[(padHeight//2):ImgPadded.shape[0] - (padHeight//2),
                (padWidth//2):ImgPadded.shape[1] - (padWidth//2)] = image
    for i in range(noRows):
        for j in range(noCols):
            imgOutput[i, j] = np.sum(
                kernel*ImgPadded[i:i+noKRows, j:j+noKCols])
    return imgOutput


In [15]:
allImgnames = os.listdir("./TrafficSigns")
mean = 0
greyScaleFilter = np.array([0.2989, 0.5870, 0.1140])
kernelBox3x3 = GenerateBoxKernel(3)
kernelBox9x9 = GenerateBoxKernel(9)
kernelBox5x5 = GenerateBoxKernel(5)
kernelgauss5x5 = GenerateGaussianKernel(5, 3)
kernelgauss9x9 = GenerateGaussianKernel(9, 5)
kernelgauss3x3 = GenerateGaussianKernel(3, 5)



for imgName in allImgnames:
    
    try:
        image = cv2.imread("./TrafficSigns/"+imgName)
        imgName_ = imgName.split('.')[0]
        imgName_ = imgName_.lower()
        imgName_ = ''.join(c for c in imgName_ if c.isalnum())
        if("plaque" in imgName_):
            imgName_ = ''.join(imgName_.split("plaque"))
        print("Working on", imgName_)

        try:
            os.mkdir('./TrafficSignDataset/'+imgName_)
        except:
            pass
        
        if("stop" in imgName_ ):
            detectedSigns = image
        else:
            detectedSigns = detectSign(image)
        i = 0
        for [x, y, w, h] in detectedSigns:
            i += 1
            boardSign = image[y:y+h, x:x+w]
            boardSign = imutils.resize(boardSign, width=500)
            greyScaleImg = convertGrayScale(boardSign, greyScaleFilter)
            SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                      "original" + str(i), boardSign)

            SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                      "greyScaleImg" + str(i), greyScaleImg)

            gaussianimgNoisy1 = AddGaussianNoise(greyScaleImg, mean, 1)
            SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                      "noice-1-" + str(i), gaussianimgNoisy1)

            saltPepperimgNoisy10 = AddSaltPepperNoise(greyScaleImg, 10)
            SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                      "noice-2-" + str(i), saltPepperimgNoisy10)

            saltPepperimgNoisy30 = AddSaltPepperNoise(greyScaleImg, 30)
            SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                      "noice-3-" + str(i), saltPepperimgNoisy30)

            boxFilter3x3ImageOnsaltPepperimgNoisy30 = Convolve(
                saltPepperimgNoisy30, kernelBox3x3)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-4-" +
                      str(i), boxFilter3x3ImageOnsaltPepperimgNoisy30)

            median3x3FilterImageOnSaltPepperimgNoisy30 = MedianFilter(
                saltPepperimgNoisy30, 3)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-5-" +
                      str(i), median3x3FilterImageOnSaltPepperimgNoisy30)

            gaussian3x3FilterImageOnSaltPepperimgNoisy30 = Convolve(
                saltPepperimgNoisy30, kernelgauss3x3)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-6-" +
                      str(i), gaussian3x3FilterImageOnSaltPepperimgNoisy30)

            boxFilter5x5ImageOnsaltPepperimgNoisy30 = Convolve(
                saltPepperimgNoisy30, kernelBox5x5)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-7-" +
                      str(i), boxFilter5x5ImageOnsaltPepperimgNoisy30)

            median5x5FilterImageOnSaltPepperimgNoisy30 = MedianFilter(
                saltPepperimgNoisy30, 5)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-8-" +
                      str(i), median5x5FilterImageOnSaltPepperimgNoisy30)

            gaussian5x5FilterImageOnSaltPepperimgNoisy30 = Convolve(
                saltPepperimgNoisy30, kernelgauss5x5)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-9-" +
                      str(i), gaussian5x5FilterImageOnSaltPepperimgNoisy30)

            boxFilter9x9ImageOnsaltPepperimgNoisy30 = Convolve(
                saltPepperimgNoisy30, kernelBox9x9)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-10-" +
                      str(i), boxFilter9x9ImageOnsaltPepperimgNoisy30)

            median9x9FilterImageOnSaltPepperimgNoisy30 = MedianFilter(
                saltPepperimgNoisy30, 9)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-11-" +
                      str(i), median9x9FilterImageOnSaltPepperimgNoisy30)

            gaussian9x9FilterImageOnSaltPepperimgNoisy30 = Convolve(
                saltPepperimgNoisy30, kernelgauss9x9)
            SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-12-" +
                      str(i), gaussian9x9FilterImageOnSaltPepperimgNoisy30)
    except:
        pass


Working on shoulderdropoffsymbol
Working on keepleftnarrow
Working on slowtrafficahead
Working on beginrightturnlaneyieldtobikes
Working on distancebetweentracksandhighway
Working on stop
Working on laneendsmergeright
Working on intersectionlanecontrol2lane
Working on leftlaneends
Working on roadworkdistance
Working on ridewithtrafficforusewithr51b
Working on dividedhighwayends
Working on allway
Working on noturns
Working on beginoneway
Working on advisoryrampspeed
Working on fine
Working on slowmovingvehicle
Working on slipperywhenwet
Working on bicyclesyieldtopeds
Working on areaclosed
Working on 
Working on shoulderdropoffforusewithw817
Working on twowaytraffic
Working on freshoiltar
Working on donotpassstoppedtrain
Working on noleftturnacrosstracksblankout
Working on yield
Working on schoolbusstopahead
Working on enddetour
Working on fallenrocks
Working on crossonlyonwalksignal
Working on doublereversecurve2lanes
Working on bridgeicesbeforeroad
Working on endoneway
Working on reduc

In [14]:
allImgnames = os.listdir("./TrafficSigns")
mean = 0
greyScaleFilter = np.array([0.2989, 0.5870, 0.1140])
kernelBox3x3 = GenerateBoxKernel(3)
kernelBox9x9 = GenerateBoxKernel(9)
kernelBox5x5 = GenerateBoxKernel(5)
kernelgauss5x5 = GenerateGaussianKernel(5, 3)
kernelgauss9x9 = GenerateGaussianKernel(9, 5)
kernelgauss3x3 = GenerateGaussianKernel(3, 5)


for imgName in allImgnames:
    image = cv2.imread("./TrafficSigns/"+imgName)
    imgName_ = imgName.split('.')[0]
    imgName_ = imgName_.lower()
    imgName_ = ''.join(c for c in imgName_ if c.isalnum())
    i += 1
    if("stop" in imgName_ or 'yield' in imgName_ ):

        boardSign = image
        boardSign = imutils.resize(boardSign, width=500)
        greyScaleImg = convertGrayScale(boardSign, greyScaleFilter)
        # SaveImage('./TrafficSignDataset/'+imgName_+ str(i), greyScaleImg)
        SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                    "original" + str(i), boardSign)

        SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                    "greyScaleImg" + str(i), greyScaleImg)

        gaussianimgNoisy1 = AddGaussianNoise(greyScaleImg, mean, 1)
        SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                    "noice-1-" + str(i), gaussianimgNoisy1)

        saltPepperimgNoisy10 = AddSaltPepperNoise(greyScaleImg, 10)
        SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                    "noice-2-" + str(i), saltPepperimgNoisy10)

        saltPepperimgNoisy30 = AddSaltPepperNoise(greyScaleImg, 30)
        SaveImage('./TrafficSignDataset/'+imgName_+"/" +
                    "noice-3-" + str(i), saltPepperimgNoisy30)

        boxFilter3x3ImageOnsaltPepperimgNoisy30 = Convolve(
            saltPepperimgNoisy30, kernelBox3x3)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-4-" +
                    str(i), boxFilter3x3ImageOnsaltPepperimgNoisy30)

        median3x3FilterImageOnSaltPepperimgNoisy30 = MedianFilter(
            saltPepperimgNoisy30, 3)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-5-" +
                    str(i), median3x3FilterImageOnSaltPepperimgNoisy30)

        gaussian3x3FilterImageOnSaltPepperimgNoisy30 = Convolve(
            saltPepperimgNoisy30, kernelgauss3x3)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-6-" +
                    str(i), gaussian3x3FilterImageOnSaltPepperimgNoisy30)

        boxFilter5x5ImageOnsaltPepperimgNoisy30 = Convolve(
            saltPepperimgNoisy30, kernelBox5x5)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-7-" +
                    str(i), boxFilter5x5ImageOnsaltPepperimgNoisy30)

        median5x5FilterImageOnSaltPepperimgNoisy30 = MedianFilter(
            saltPepperimgNoisy30, 5)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-8-" +
                    str(i), median5x5FilterImageOnSaltPepperimgNoisy30)

        gaussian5x5FilterImageOnSaltPepperimgNoisy30 = Convolve(
            saltPepperimgNoisy30, kernelgauss5x5)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-9-" +
                    str(i), gaussian5x5FilterImageOnSaltPepperimgNoisy30)

        boxFilter9x9ImageOnsaltPepperimgNoisy30 = Convolve(
            saltPepperimgNoisy30, kernelBox9x9)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-10-" +
                    str(i), boxFilter9x9ImageOnsaltPepperimgNoisy30)

        median9x9FilterImageOnSaltPepperimgNoisy30 = MedianFilter(
            saltPepperimgNoisy30, 9)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-11-" +
                    str(i), median9x9FilterImageOnSaltPepperimgNoisy30)

        gaussian9x9FilterImageOnSaltPepperimgNoisy30 = Convolve(
            saltPepperimgNoisy30, kernelgauss9x9)
        SaveImage('./TrafficSignDataset/'+imgName_+"/"+"noice-12-" +
                    str(i), gaussian9x9FilterImageOnSaltPepperimgNoisy30)

